In [63]:
import os
import shutil
import sys
import subprocess
import requests
import json
import re
import time
import yt_dlp

DEBUG = 1
wallpaper_engine_appid = 431960
BASE_DIR = '/Users/bebeal/Programs/ProjectionSorcery'
LOCAL_WORKSHOP_DIR = './cache/workshop'
YOUTUBE_DOWNLOAD_DIR = './cache/youtube'
REDDIT_DOWNLOAD_DIR = './cache/reddit'
IMGUR_DOWNLOAD_DIR = './cache/imgur'

def debug_print(level, message):
    if DEBUG >= level:
        print(message)

def ensure_directory_exists(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

def check_cache(directory, filename):
    file_path = os.path.join(directory, filename)
    if os.path.exists(file_path):
        debug_print(1, f'Cache hit at {directory}/{filename}')
        return file_path
    return None

In [64]:
######################################
#             Youtube                #
######################################

youtube_domain = 'youtube.com'
def youtube_video(video_id): return f'https://www.{youtube_domain}/watch?v={video_id}'
def youtube_playlist(playlist_id): return f'https://www.{youtube_domain}/playlist?list={playlist_id}'
def youtube_channel(channel_id): return f'https://www.{youtube_domain}/channel/{channel_id}'
def youtube_search(query): return f'https://www.{youtube_domain}/results?search_query={query}'
def youtube_channel_videos(channel_id): return f'https://www.{youtube_domain}/channel/{channel_id}/videos'

def download_youtube_video(video_id, dir=YOUTUBE_DOWNLOAD_DIR):
  ensure_directory_exists(dir)
  if not check_cache(dir, f'{video_id}.mp4'):
    # https://github.com/ytdl-org/youtube-dl/blob/master/youtube_dl/YoutubeDL.py#L157
    ydl_opts = {
      'outtmpl': f'{dir}/{video_id}',
      'format': "bv*[ext=mp4]+ba[ext=m4a]/b[ext=mp4] / bv*+ba/b" # Download the best mp4 video available, or the best video if no mp4 available
    }
    url = f'https://www.{youtube_domain}/watch?v={video_id}'
    yt_dlp.YoutubeDL(ydl_opts).download([url])
  return f'{dir}/{video_id}.mp4'
# download_youtube_video('YAgjfMR9R_M')

In [65]:
######################################
#             Reddit                 #
######################################

reddit_domain = 'reddit.com'
reddit_content_domain = 'i.redd.it'
def reddit_post(post_id): return f'https://www.{reddit_domain}/comments/{post_id}/.json'
def reddit_subreddit(subreddit): return f'https://www.{reddit_domain}/r/{subreddit}/.json'
def reddit_search(query, domain=reddit_domain): return f'https://www.{domain}/search/.json?q={query}'

def download_reddit_content(content, dir=REDDIT_DOWNLOAD_DIR):
  ensure_directory_exists(dir)
  if not check_cache(dir, content):
    url = f'https://{reddit_content_domain}/{content}'
    response = requests.get(url)
    if response.status_code == 200:
      file = open(os.path.join(dir, content), 'wb')
      file.write(response.content)
      debug_print(1, f'Downloaded {url} to {dir}/{content}')
      file.close()
    else:
      debug_print(1, f'Failed to download {url}')
  return f'{dir}/{content}'

# download_reddit_content('xv7ooelxqbu61.png')

In [66]:
######################################
#             Steam                  #
######################################

steam_domain = 'steamcommunity.com'
def steam_workshop(workshop_id): return f'https://steamcommunity.com/sharedfiles/filedetails/?id={workshop_id}'
login = f"{os.environ['STEAM_USER']} {os.environ['STEAM_PASS']}" # ' anonymous' for anonymous login

def run_steamcmd(command):
    base_command = f"steamcmd +force_install_dir {BASE_DIR} +login {login}"
    full_command = f"{base_command} {command} +quit"
    result = subprocess.run(full_command, shell=True)
    return result.returncode

def check_steam_cache(dir, workshop_id):
    # check for any extension
    for ext in ['jpg', 'jpeg', 'png', 'mp4', 'webm', 'gif', 'pkg', 'json']:
        if check_cache(dir, f'{workshop_id}.{ext}'):
            return ext

def download_steam_workshop_item(workshop_id, app_id=wallpaper_engine_appid, dir=LOCAL_WORKSHOP_DIR):
    if not workshop_id.isnumeric():
        only_numeric = re.sub(r'\D', '', workshop_id)
        debug_print(1, f'Invalid workshop ID {workshop_id}, using {only_numeric}')
        workshop_id = only_numeric
    ensure_directory_exists(dir)
    extension_found = check_steam_cache(dir, workshop_id)
    if not extension_found:
        run_steamcmd(f'+workshop_download_item {app_id} {workshop_id}')
        workshop_dir = f'steamapps/workshop/content/{app_id}/{workshop_id}'
        if os.path.exists(workshop_dir):
            project_file = open(os.path.join(workshop_dir, 'project.json'), 'r')
            project = json.load(project_file)
            project_name = project['file']
            extension = project_name.split('.')[-1]
            project_name = project_name.replace('.' + extension, '')
            if project_name:  # Check if project_name is not None or empty
                if extension == 'json':
                    extension = 'pkg'
                source_path = os.path.join(workshop_dir, f'{project_name}.{extension}')
                destination_path = os.path.join(dir, f'{workshop_id}.{extension}')
                shutil.move(source_path, destination_path)
                return destination_path
    return f'{dir}/{workshop_id}.{extension_found}'

def clean_steamcmd():
    shutil.rmtree(f'steamapps', ignore_errors=True)

def clear_cache():
    shutil.rmtree(LOCAL_WORKSHOP_DIR, ignore_errors=True)

download_steam_workshop_item('3257943478')
# download_steam_workshop_item('2412473886')
# clean_steamcmd()

Cache hit at ./cache/workshop/3257943478.mp4


'./cache/workshop/3257943478.mp4'